In [1]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

import numpy as np
import tensorflow as tf

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from numpy import array
from numpy import argmax



categories = [
        'alt.atheism',
        'talk.religion.misc',
        'rec.autos',
        'talk.politics.guns',
        'rec.sport.baseball'
    ]

remove = ()

print("Loading 20 newsgroups dataset for categories:")
print("\t\t" + str(categories))


data_train = fetch_20newsgroups(subset='train', categories=categories,
                                shuffle=True, random_state=42,
                                remove=remove)

data_test = fetch_20newsgroups(subset='test', categories=categories,
                               shuffle=True, random_state=42,
                               remove=remove)



print("train and test labes are being encoded to onehot vector")

target_names = data_train.target_names
y_train, y_test = data_train.target, data_test.target

print("integer classes")

print("\ttrain")
print(y_train)

print("\ttest")
print(y_test)


label_encoder_train = LabelEncoder()
onehot_encoder_train = OneHotEncoder(sparse=False)

integer_encoded_train = label_encoder_train.fit_transform(y_train)
integer_encoded_train = integer_encoded_train.reshape(len(integer_encoded_train), 1)

y_train_onehot = onehot_encoder_train.fit_transform(integer_encoded_train)


label_encoder_test = LabelEncoder()
onehot_encoder_test = OneHotEncoder(sparse=False)


integer_encoded_test = label_encoder_test.fit_transform(y_test)
integer_encoded_test = integer_encoded_test.reshape(len(integer_encoded_test), 1)

y_test_onehot = onehot_encoder_test.fit_transform(integer_encoded_test)


print("encoded clasesses")
print("\ttrain")
print(y_train_onehot)
print("\ttest")
print(y_test_onehot)



print("Extracting features from the training data using a sparse vectorizer")
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5, stop_words='english')
X_train = vectorizer.fit_transform(data_train.data)

print("n_samples: %d, n_features: %d" % X_train.shape)


print("Extracting features from the test data using the same vectorizer")
X_test = vectorizer.transform(data_test.data)
print("n_samples: %d, n_features: %d" % X_test.shape)

X_train = X_train.toarray()
X_test = X_test.toarray()

feature_flat_size = X_train[0].size
num_classes = len(target_names)


print("Size of:")
print("- Training-set:\t\t\t{}".format(len(X_train)))
print("- Test-set:\t\t\t{}".format(len(X_test)))

#print(str(X_train))
if(X_train[0].size == X_test[0].size):
    print("- Feature:\t\t\t{}".format(str(X_train[0].size)))
#print("feature size test: " + str(X_test[0].size))

print("- Number of classes:\t\t{}".format(str(num_classes)))

Loading 20 newsgroups dataset for categories:
		['alt.atheism', 'talk.religion.misc', 'rec.autos', 'talk.politics.guns', 'rec.sport.baseball']
train and test labes are being encoded to onehot vector
integer classes
	train
[1 1 2 ..., 1 2 3]
	test
[4 4 2 ..., 1 3 3]
encoded clasesses
	train
[[ 0.  1.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 ..., 
 [ 0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 0.  0.  0.  1.  0.]]
	test
[[ 0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.]
 [ 0.  0.  1.  0.  0.]
 ..., 
 [ 0.  1.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.]
 [ 0.  0.  0.  1.  0.]]
Extracting features from the training data using a sparse vectorizer
n_samples: 2594, n_features: 37364
Extracting features from the test data using the same vectorizer
n_samples: 1727, n_features: 37364
Size of:
- Training-set:			2594
- Test-set:			1727
- Feature:			37364
- Number of classes:		5


In [2]:
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

def new_biases(length):
    return tf.Variable(tf.constant(0.05, shape=[length]))


In [3]:
'''
    uses 5x1 filter
    number of channel is 1
    poolin is 2x1

'''

def new_conv_layer(input,              # The previous layer.
                   num_input_channels,
                   num_filters,        # Number of filters.
                   use_pooling=True):  # Use 2x2 max-pooling.

    # Shape of the filter-weights for the convolution.
    # This format is determined by the TensorFlow API.
    shape = [5, 1, num_input_channels, num_filters]

    # Create new weights aka. filters with the given shape.
    weights = new_weights(shape=shape)

    # Create new biases, one for each filter.
    biases = new_biases(length=num_filters)

    # Create the TensorFlow operation for convolution.
    # Note the strides are set to 1 in all dimensions.
    # The first and last stride must always be 1,
    # because the first is for the image-number and
    # the last is for the input-channel.
    # But e.g. strides=[1, 2, 2, 1] would mean that the filter
    # is moved 2 pixels across the x- and y-axis of the image.
    # The padding is set to 'SAME' which means the input image
    # is padded with zeroes so the size of the output is the same.
    layer = tf.nn.conv2d(input=input,
                         filter=weights,
                         strides=[1, 1, 1, 1],
                         padding='SAME')

    # Add the biases to the results of the convolution.
    # A bias-value is added to each filter-channel.
    layer += biases

    # Use pooling to down-sample the image resolution?
    if use_pooling:
        # This is 2x2 max-pooling, which means that we
        # consider 2x2 windows and select the largest value
        # in each window. Then we move 2 pixels to the next window.
        layer = tf.nn.max_pool(value=layer,
                               ksize=[1, 2, 1, 1],
                               strides=[1, 2, 1, 1],
                               padding='SAME')

    # Rectified Linear Unit (ReLU).
    # It calculates max(x, 0) for each input pixel x.
    # This adds some non-linearity to the formula and allows us
    # to learn more complicated functions.
    layer = tf.nn.relu(layer)

    # Note that ReLU is normally executed before the pooling,
    # but since relu(max_pool(x)) == max_pool(relu(x)) we can
    # save 75% of the relu-operations by max-pooling first.

    # We return both the resulting layer and the filter-weights
    # because we will plot the weights later.
    return layer, weights

In [4]:
'''
A convolutional layer produces an output tensor with 4 dimensions. 
We will add fully-connected layers after the convolution layers, 
so we need to reduce the 4-dim tensor to 2-dim which can be used 
as input to the fully-connected layer.
'''

def flatten_layer(layer):
    # Get the shape of the input layer.
    layer_shape = layer.get_shape()

    # The shape of the input layer is assumed to be:
    # layer_shape == [num_documets, img_height, img_width, num_channels]

    # The number of features is: img_height * img_width * num_channels
    # We can use a function from TensorFlow to calculate this.
    num_features = layer_shape[1:4].num_elements()
    
    # Reshape the layer to [num_images, num_features].
    # Note that we just set the size of the second dimension
    # to num_features and the size of the first dimension to -1
    # which means the size in that dimension is calculated
    # so the total size of the tensor is unchanged from the reshaping.
    layer_flat = tf.reshape(layer, [-1, num_features])

    # The shape of the flattened layer is now:
    # [num_images, img_height * img_width * num_channels]

    # Return both the flattened layer and the number of features.
    return layer_flat, num_features

In [5]:
def new_fc_layer(input,          # The previous layer.
                 num_inputs,     # Num. inputs from prev. layer.
                 num_outputs,    # Num. outputs.
                 use_relu=True): # Use Rectified Linear Unit (ReLU)?

    # Create new weights and biases.
    weights = new_weights(shape=[num_inputs, num_outputs])
    biases = new_biases(length=num_outputs)

    # Calculate the layer as the matrix multiplication of
    # the input and weights, and then add the bias-values.
    layer = tf.matmul(input, weights) + biases

    # Use ReLU?
    if use_relu:
        layer = tf.nn.relu(layer)

    return layer

In [6]:
x = tf.placeholder(tf.float32, shape=[None, feature_flat_size], name='x')
x_doc = tf.reshape(x, [-1, feature_flat_size, 1, 1])

y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')


y_true_cls = tf.argmax(y_true, dimension=1)




Instructions for updating:
Use the `axis` argument instead


In [7]:
#creating first conv. layer by using 16 filters and pooling

layer_conv1, weights_conv1 = \
    new_conv_layer(input=x_doc,
                   num_input_channels = 1,
                   num_filters=16,
                   use_pooling=True)

In [8]:
layer_conv1

<tf.Tensor 'Relu:0' shape=(?, 18682, 1, 16) dtype=float32>

In [9]:
feature_flat_size

37364

In [11]:
#creating second conv. layer by using 36 filters and pooling

layer_conv2, weights_conv2 = \
    new_conv_layer(input=layer_conv1,
                   num_input_channels = 16,
                   num_filters=36,
                   use_pooling=True)

In [12]:
layer_flat, num_features = flatten_layer(layer_conv2)



In [13]:
layer_flat

<tf.Tensor 'Reshape_1:0' shape=(?, 336276) dtype=float32>

In [14]:
num_features

336276

In [15]:
#creating fully connected layer with 128 number of nodes

layer_fc1 = new_fc_layer(input=layer_flat,
                         num_inputs=num_features,
                         num_outputs=128, 
                         use_relu=True)

In [16]:
layer_fc1

<tf.Tensor 'Relu_2:0' shape=(?, 128) dtype=float32>

In [20]:
#Add another fully-connected layer that outputs vectors of length num_classes
#for determining which of the num_classes classes the input image belongs to. 
#Note that ReLU is not used in this layer.

layer_fc2 = new_fc_layer(input=layer_fc1,
                         num_inputs=128,
                         num_outputs=num_classes,
                         use_relu=False)

In [21]:
layer_fc2

<tf.Tensor 'add_4:0' shape=(?, 5) dtype=float32>

In [22]:
y_pred = tf.nn.softmax(layer_fc2)


In [23]:
y_pred_cls = tf.argmax(y_pred, dimension=1)


Instructions for updating:
Use the `axis` argument instead


In [24]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2,
                                                        labels=y_true)


In [25]:
cost = tf.reduce_mean(cross_entropy)


In [26]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

In [27]:
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [28]:

session = tf.Session()

session.run(tf.global_variables_initializer())


In [29]:
def optimize(num_iterations):
    feed_dict_train = {x: X_train, y_true: y_train_onehot}
    
    for i in range(num_iterations):
        # Put the batch into a dict with the proper names
        # for placeholder variables in the TensorFlow graph.
        # Note that the placeholder for y_true_cls is not set
        # because it is not used during training.
        
        # Run the optimizer using this batch of training data.
        # TensorFlow assigns the variables in feed_dict_train
        # to the placeholder variables and then runs the optimizer.
        session.run(optimizer, feed_dict=feed_dict_train)
        
        if i % 100 == 0:
            # Calculate the accuracy on the training-set.
            acc = session.run(accuracy, feed_dict=feed_dict_train)

            # Message for printing.
            msg = "Optimization Iteration: {0:>6}, Training Accuracy: {1:>6.1%}"

            # Print it.
            print(msg.format(i + 1, acc))
            
        

In [30]:
feed_dict_test = {x: X_test,
                  y_true: y_test_onehot,
                  y_true_cls: y_test}

In [31]:
def print_accuracy():
    # Use TensorFlow to compute the accuracy.
    acc = session.run(accuracy, feed_dict=feed_dict_test)
    
    # Print the accuracy.
    print("Accuracy on test-set: {0:.1%}".format(acc))

In [32]:
print_accuracy()
#this is no optimization

Accuracy on test-set: 22.9%


In [ ]:
optimize(num_iterations=1000)
